# 0. Standard imports

In [165]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('WebAgg')
import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
cd ..

/home/eecs/khalil.ouardini/Cassiopeia_Transcriptome/scvi/external


In [3]:
cd ..

/home/eecs/khalil.ouardini/Cassiopeia_Transcriptome/scvi


***import ete3 Tree***

In [4]:
from ete3 import Tree

#tree_name = "/Users/khalilouardini/Desktop/projects/scVI/scvi/data/Cassiopeia_trees/3726_NT_T1_tree.processed.collapsed.tree"
tree_name = "/home/eecs/khalil.ouardini/cas_scvi_topologies/Cassiopeia_trees/tree_test.txt"
tree = Tree(tree_name, 1)

#tree = Tree()
#tree.populate(60)

for i, n in enumerate(tree.traverse('levelorder')):
    n.add_features(index=i)

***Prune tree***

In [5]:
for i, n in enumerate(tree.traverse()):
    print(i, n.name)

split_node = tree.search_nodes(name='2|2|2|2|2|0|2|2|2|0|2|2|0|2|0|2|2|2|0|2|2|2|2|0|0|0|0|0|0')[0]
sub_leaves = [n.name for n in split_node.get_leaves()]
tree.prune(sub_leaves)

GCCTGAG-1
443 L6.AACGTTGCATGAACCT-1
444 L6.CGTCACTCATCGGTTA-1
445 L6.TGGCCAGCAGTAACGG-1
446 L6.GCTGGGTCATACGCTA-1
447 L6.CGCTTCATCAAACAAG-1
448 L6.CATGGCGGTTAAGATG-1
449 L6.GTCTCGTAGTGAAGTT-1
450 L6.GAAATGAGTTGGTGGA-1
451 L6.TCATTACTCAGTACGT-1
452 L6.TTCTACAAGGCATGTG-1
453 4|2|2|2|2|9|2|7|2|2|2|5|0|6|5|-|-|-|5|6|5|2|11|-|-|-|-|-|-
454 4|2|2|2|2|9|2|7|2|2|2|5|2|6|0|6|2|0|5|6|5|2|8|3|0|24|-|-|-
455 5|2|2|2|2|5|2|7|2|2|2|5|8|8|9|4|2|2|9|6|5|2|8|3|12|10|-|-|-
456 L6.TGGCCAGCAATCACAC-1
457 L6.CACAGTACAGACACTT-1
458 L6.TATCAGGTCGAGGTAG-1
459 L6.TGGGAAGAGGGAACGG-1
460 L6.CATCAAGAGTTCGCGC-1
461 L6.TTGGAACGTTATGTGC-1
462 L6.CACAGGCTCGATGAGG-1
463 -|-|-|2|2|5|5|6|2|2|2|5|5|5|5|5|2|2|-|-|-|2|8|3|7|2|-|-|-
464 L6.GGGACCTCAGACAAAT-1
465 L6.GCAATCAGTCCAGTTA-1
466 L6.ACACCGGGTCCGTTAA-1
467 L6.GTGTTAGCAGCGATCC-1
468 L6.CGATGGCAGGCGATAC-1
469 L6.GCAGTTATCATTGCGA-1
470 -|-|-|2|2|2|-|-|-|2|2|2|2|2|2|2|2|2|2|2|2|2|2|-|-|-|-|-|-
471 L6.TGACAACTCAGCTCGG-1
472 L6.CCAGCGACACGCATCG-1
473 L6.CGCTATCAGGGTCGAT-1


In [181]:
# Data
from anndata import AnnData
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from external.dataset.tree import TreeDataset, GeneExpressionDataset
from external.dataset.poisson_glm import Poisson_GLM
from external.dataset.anndataset import AnnDatasetFromAnnData

# Models
from models import *;
import scanpy as sc
from external.inference.tree_inference import TreeTrainer
from inference.inference import UnsupervisedTrainer
from scvi.inference import posterior
from external.models.treevae import TreeVAE

# Utils
from external.utils.data_util import get_leaves, get_internal
from external.utils.metrics import ks_pvalue, accuracy_imputation, correlations, knn_purity, knn_purity_stratified
from external.utils.plots_util import plot_histograms, plot_scatter_mean, plot_ecdf_ks, plot_density
from external.utils.plots_util import plot_losses, plot_elbo, plot_common_ancestor, plot_one_gene, training_dashboard
from external.utils.baselines import avg_weighted_baseline, scvi_baseline, scvi_baseline_z, cascvi_baseline_z

# 1. Simulations (Poisson GLM)

In [12]:
d = 5
g = 100
vis = False
leaves_only = False
var = 1.0

glm = Poisson_GLM(tree, g, d, vis, leaves_only, var)

glm.simulate_latent()

***Generate gene expression count data***

In [13]:
glm.simulate_ge()
# Quality Control (i.e Gene Filtering)
glm.gene_qc()
glm.X.shape

(474, 100)

***GLM parameters***

In [14]:
glm.W.shape, glm.beta.shape

((100, 5), (100,))

***Binomial thinning***

In [15]:
print("Proportion of dropouts: {}".format(np.mean(glm.X == 0)))
glm.binomial_thinning(p=0.01)
print("Proportion of dropouts after Binomial thinning: {}".format(np.mean(glm.X == 0)))

Proportion of dropouts: 0.4418565400843882
Proportion of dropouts after Binomial thinning: 0.9072995780590717


Get the data and the indexes at the leaves

In [16]:
# Latent vectors
leaves_z, _, _ = get_leaves(glm.z, glm.mu, tree)

#FIXED training set
leaves_X, leaves_idx, mu = get_leaves(glm.X, glm.mu, tree)

# internal nodes data (for imputation)
internal_X, internal_idx, internal_mu = get_internal(glm.X, glm.mu, tree)

leaves_X.shape, mu.shape, internal_X.shape, internal_mu.shape, leaves_z.shape

((402, 100), (402, 100), (72, 100), (72, 100), (402, 5))

***Visualizing the data***

***Tree***

In [18]:
#plot_one_gene(tree=tree,
#             X=glm.X,
#             g=0,
#             node_sizes=[800 for i in range(glm.X.shape[0])],
#             var='latent',
#             size=1000,
#             show_index=True,
#             save_fig=True,
#             figsize=(180, 100)
#           )

***Simulated latent space***

In [19]:
plot_common_ancestor(tree,
                     glm.z,
                     embedding='umap',
                     give_labels=False)

# 2. Fitting CascVI

In [93]:
import scanpy as sc

# anndata + gene and celle filtering
adata = AnnData(leaves_X)
leaves = [n for n in tree.traverse('levelorder') if n.is_leaf()]
adata.obs_names = [n.name for n in leaves]
#sc.pp.filter_genes(adata, min_counts=3)
#sc.pp.filter_cells(adata, min_counts=0)

In [94]:
scvi_dataset = AnnDatasetFromAnnData(adata, filtering=False)
scvi_dataset.initialize_cell_attribute('barcodes', adata.obs_names)

go
[2021-05-02 23:19:37,121] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2021-05-02 23:19:37,123] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2021-05-02 23:19:37,124] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]


***Create a TreeDataset object***

In [96]:
# treeVAE
import copy

tree_bis = copy.deepcopy(tree)
cas_dataset = TreeDataset(scvi_dataset, tree=tree_bis, filtering=False)
cas_dataset

# No batches beacause of the message passing
use_cuda = False
use_MP = True

[2021-05-02 23:19:53,953] INFO - scvi.dataset.dataset | Merging datasets. Input objects are modified in place.
[2021-05-02 23:19:53,954] INFO - scvi.dataset.dataset | Gene names and cell measurement names are assumed to have a non-null intersection between datasets.
[2021-05-02 23:19:53,956] INFO - scvi.dataset.dataset | Keeping 100 genes
[2021-05-02 23:19:53,958] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2021-05-02 23:19:53,960] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2021-05-02 23:19:53,961] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2021-05-02 23:19:53,962] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2021-05-02 23:19:53,965] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2021-05

***Initialize model***

In [118]:
treevae = TreeVAE(cas_dataset.nb_genes,
              tree = cas_dataset.tree,
              n_latent=glm.latent,
              n_hidden=128,
              n_layers=1,
              reconstruction_loss='poisson',
              prior_t = var,
              ldvae = True,
              use_MP=True
             )

In [119]:
import torch

new_weight = torch.from_numpy(glm.W).float()
new_bias = torch.from_numpy(glm.beta).float()

with torch.no_grad():
    treevae.decoder.factor_regressor.fc_layers[0][0].weight = torch.nn.Parameter(new_weight)
    treevae.decoder.factor_regressor.fc_layers[0][0].bias = torch.nn.Parameter(new_bias)
    
for param in treevae.decoder.factor_regressor.fc_layers[0][0].parameters():
    param.requires_grad = False
    
treevae.decoder

LinearDecoder(
  (factor_regressor): FCLayers(
    (fc_layers): Sequential(
      (Layer 0): Sequential(
        (0): Linear(in_features=5, out_features=100, bias=True)
        (1): None
        (2): None
        (3): None
      )
    )
  )
)

In [120]:
assert(treevae.decoder.factor_regressor.fc_layers[0][0].weight.numpy().all() == glm.W.T.all())
assert(treevae.decoder.factor_regressor.fc_layers[0][0].bias.numpy().all() == glm.beta.all())

***Are we able to generate the gene expression data by decoding the simulated latent space?***

In [121]:
px_scale, px_rate, raw_px_scale = treevae.decoder(treevae.dispersion,
                                        torch.from_numpy(leaves_z).float(),
                                        torch.from_numpy(np.array([np.log(10000)])).float()
                                        )

In [122]:
from sklearn.metrics import mean_squared_error

foo = np.clip(a=np.exp(raw_px_scale.detach().cpu().numpy()),
              a_min=0,
              a_max=5000
)
mse = mean_squared_error(mu, foo)
print("the distance between the Poisson and the NB means is {}".format(mse))

the distance between the Poisson and the NB means is 1.1905326647433946e-08


***Hyperparameters***

In [123]:
n_epochs = 400
lr = 1e-3
lambda_ = 1.0

***trainer***

In [124]:
freq = 100
trainer = TreeTrainer(
    model = treevae,
    gene_dataset = cas_dataset,
    lambda_ = lambda_,
    train_size=1.0,
    test_size=0,
    use_cuda=use_cuda,
    frequency=freq,
    n_epochs_kl_warmup=None
)

train_leaves:  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119], [120], [121], [122], [123], [124], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [135], [136], [137], [138], [139], [140], [141], [142], [143], [144], [145], [146], [147], [148], [149], [150], [151], [152], [153], [154], [155], [1

***Start training***

In [125]:
trainer.train(n_epochs=n_epochs,
              lr=lr
              )

996459227151
Varitional Likelihood: 1.1591878103379587
ELBO Loss: 135.14704753138102
training:  77%|███████▋  | 307/400 [05:52<01:38,  1.06s/it]Reconstruction Loss: 128.1396047108209
Encodings MP Likelihood: 5.668449224874802
Varitional Likelihood: 1.2635177688219061
ELBO Loss: 135.0715717045176
training:  77%|███████▋  | 308/400 [05:53<01:37,  1.06s/it]Reconstruction Loss: 127.95967428482587
Encodings MP Likelihood: 5.67236822908187
Varitional Likelihood: 1.3484676228233832
ELBO Loss: 134.9805101367311
training:  77%|███████▋  | 309/400 [05:54<01:35,  1.05s/it]Reconstruction Loss: 128.01919115360695
Encodings MP Likelihood: 5.691763664636124
Varitional Likelihood: 1.1286088174848414
ELBO Loss: 134.83956363572793
training:  78%|███████▊  | 310/400 [05:55<01:43,  1.15s/it]Reconstruction Loss: 127.8945992692786
Encodings MP Likelihood: 5.6656756097959375
Varitional Likelihood: 1.1863814776216572
ELBO Loss: 134.7466563566962
training:  78%|███████▊  | 311/400 [05:57<01:49,  1.24s/it]Recon

***Loss Functions***

In [208]:
training_dashboard(trainer, treevae.encoder_variance)

### 3. Posterior and MV imputation

In [135]:
full_posterior = trainer.create_posterior(trainer.model, cas_dataset, trainer.clades,
                                indices=np.arange(len(cas_dataset))
                                         )

***Missing Value imputation By Posterior Predictive sampling***

In [136]:
empirical_l = np.mean(np.sum(glm.X, axis=1))

# CascVI impitations
imputed = {}
imputed_z = {}
imputed_gt = {}

for n in tree.traverse('levelorder'):
    if not n.is_leaf():
        imputed[n.name], imputed_z[n.name] = full_posterior.imputation_internal(n.name,
                                                            give_mean=False,
                                                            library_size=empirical_l
                                                           )
        imputed_gt[n.name] = glm.X[n.index]

In [137]:
imputed_X = [x for x in imputed.values()]
imputed_X = np.array(imputed_X).reshape(-1, cas_dataset.X.shape[1])
#plot_histograms(imputed_X, "Histogram of CasscVI imputed gene expression data")

***CascVI Baseline 1 (Average of reconstructions)***

In [138]:
#imputed_cascvi_1 = scvi_baseline(tree, full_posterior, True)

***CascVI Baseline 2 (Reconstruction of Averaged latent space)***

In [173]:
imputed_cascvi_2, imputed_cascvi_2_z = scvi_baseline_z(tree=tree,
                                   model=treevae,
                                   posterior=full_posterior,
                                   weighted=True,
                                   n_samples_z=1,
                                   library_size=empirical_l,
                                   gaussian=False
                                  )

# 4. Baselines

***Baseline 1: Unweighted Average of gene expression in Clade***

The simple idea here is to impute the value of an internal node, with the (un)weighted average of the gene expression values of the leaves, taking the query internal node as the root of the subtree.

In [141]:
weighted = True
imputed_avg = avg_weighted_baseline(tree, True, glm.X, rounding=True)

#get internal nodes
avg_X = np.array([x for x in imputed_avg.values()]).reshape(-1, glm.X.shape[1])
internal_avg_X, _, _ = get_internal(avg_X, glm.mu, tree)
#plot_histograms(avg_X,
#               'Histograms of imputed GE values with the average baseline')

***Baseline 2: (Un)weighted Average of decoded latent vectors, with scVI***

We use the same averaging of the subtrees leaves in **Baseline 1**, only this time, the gene expression data is recovered with scVI

In [142]:
# anndata
gene_dataset = GeneExpressionDataset()
gene_dataset.populate_from_data(leaves_X)

[2021-05-02 23:59:01,561] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2021-05-02 23:59:01,564] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2021-05-02 23:59:01,565] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]


In [184]:
import torch

n_epochs = 400
use_batches = False

vae = VAE(gene_dataset.nb_genes,
                  n_batch=cas_dataset.n_batches * use_batches,
                  n_hidden=128,
                  n_layers=1,
                  reconstruction_loss='poisson',
                  n_latent=glm.latent,
                  ldvae=True
              )


new_weight = torch.from_numpy(glm.W).float()
new_bias = torch.from_numpy(glm.beta).float()

with torch.no_grad():
    vae.decoder.factor_regressor.fc_layers[0][0].weight = torch.nn.Parameter(new_weight)
    vae.decoder.factor_regressor.fc_layers[0][0].bias = torch.nn.Parameter(new_bias)
    
for param in vae.decoder.factor_regressor.fc_layers[0][0].parameters():
    param.requires_grad = False
    

==== We will use a linear decoder ===


In [185]:
trainer_scvi = UnsupervisedTrainer(model=vae,
                              gene_dataset=gene_dataset,
                              train_size=1.0,
                              use_cuda=use_cuda,
                              frequency=10,
                              n_epochs_kl_warmup=None)

# train scVI
trainer_scvi.train(n_epochs=n_epochs, lr=1e-3) 
                                        
elbo_train_scvi = trainer_scvi.history["elbo_train_set"]
x = np.linspace(0, 100, (len(elbo_train_scvi)))
plt.plot(np.log(elbo_train_scvi), 
         label="train", color='blue',
         linestyle=':',
         linewidth=3
        )
plt.xlabel('Epoch')
plt.ylabel("ELBO")
plt.legend()
plt.title("Train history scVI")
plt.show()

training: 100%|██████████| 400/400 [00:16<00:00, 23.84it/s]
Press Ctrl+C to stop WebAgg server


RuntimeError: This event loop is already running

***scVI Baseline 1 (Average of reconstructions)***

In [ ]:
scvi_posterior = trainer_scvi.create_posterior(model=vae,
                                              gene_dataset=gene_dataset
                                              )

#imputed_scvi, imputed_scvi_z = scvi_baseline(tree=tree, 
#                                             posterior=scvi_posterior, 
#                                             weighted=True,
#                                            give_latent=True,
#                                            n_samples_z=1)

***scVI Baseline 2 (Decoded Average Latent space)***

In [189]:
library_size = np.mean(np.sum(glm.X, axis=1))
scvi_latent = np.array([scvi_posterior.get_latent(give_mean=False)[0] for i in range(10)])

imputed_scvi_2, imputed_scvi_2_z = scvi_baseline_z(tree=tree,
                                 model=vae,
                                 posterior=scvi_posterior,
                                 weighted=True,
                                 n_samples_z=1,
                                 library_size=library_size,
                                 known_latent=True,
                                 latent=scvi_latent,
                                 gaussian=False
                                )


AssertionError: nb. categorical args provided doesn't match init. params.

# 5. Likelihood Ratio

In [190]:
cascvi_latent = full_posterior.get_latent()
scvi_latent = scvi_posterior.get_latent()[0]

scvi_latent.shape, cascvi_latent.shape

((402, 5), (402, 5))

In [192]:
treevae.initialize_visit()
treevae.initialize_messages(scvi_latent, cas_dataset.barcodes, scvi_latent.shape[1])
treevae.perform_message_passing((treevae.tree & treevae.root), scvi_latent.shape[1], False)
mp_lik_scvi = treevae.aggregate_messages_into_leaves_likelihood(d, add_prior=True)
print("Likelihood of scVI encodings: ", mp_lik_scvi.item())

Likelihood of scVI encodings:  -2834.7010432824986


In [195]:
treevae.initialize_visit()
treevae.initialize_messages(cascvi_latent, cas_dataset.barcodes, cascvi_latent.shape[1])
treevae.perform_message_passing((treevae.tree & treevae.root), cascvi_latent.shape[1], False)
mp_lik_cascvi = treevae.aggregate_messages_into_leaves_likelihood(d, add_prior=True)
print("Likelihood of cascVI encodings: ", mp_lik_cascvi.item())

Likelihood of cascVI encodings:  -13622.112278252629


In [196]:
# Likelihood ratio
lambda_ = (mp_lik_cascvi - mp_lik_scvi)
print("Likelihood Ratio:", lambda_)

Likelihood Ratio: tensor(-10787.4112, dtype=torch.float64)


# 6. Latent Space Analysis

***CascVI***

In [ ]:
plot_common_ancestor(tree,
                     cascvi_latent,
                     embedding='umap',
                     give_labels=False
                             )

In [ ]:
# Merge internal nodes and leaves
full_cascvi_latent = construct_latent(tree, cascvi_latent, imputed_z)


print("CascVI latent space")
plot_common_ancestor(tree,
                     full_cascvi_latent,
                     embedding='umap',
                     give_labels=False
                             )



In [ ]:
full_cascvi_latent_2 = construct_latent(tree, cascvi_latent, imputed_cascvi_2_z)

print("CascVI + averaging latent space")
plot_common_ancestor(tree,
                     full_cascvi_latent_2,
                     embedding='umap',
                     give_labels=False
                             )

***scVI***

In [ ]:
# Merge internal nodes and leaves
full_scvi_latent = construct_latent(tree, scvi_latent, imputed_scvi_2_z)

print("scVI latent space")
plot_common_ancestor(tree,
                 full_scvi_latent,
                 embedding='umap',
                 give_labels=False
                    )

***k-NN purity***

Leaves only

In [ ]:
print("Leaves Only")
scores = knn_purity(max_neighbors=50,
              data=[leaves_z, scvi_latent, cascvi_latent],
              plot=True)

Internal nodes only

In [ ]:
print("Internal nodes Only")
internal_z, internal_idx, internal_mu = get_internal(glm.z, glm.mu, tree)
internal_scvi_z, _, _ = get_internal(full_scvi_latent, glm.mu, tree)
internal_scvi_z_2, _, _ = get_internal(full_scvi_latent_2, glm.mu, tree)
internal_cascvi_z, _, _ = get_internal(full_cascvi_latent, glm.mu, tree)
internal_cascvi_z_2, _, _ = get_internal(full_cascvi_latent_2, glm.mu, tree)

scores = knn_purity(max_neighbors=50,
              data=[internal_z, internal_scvi_z, internal_scvi_z_2, internal_cascvi_z, internal_cascvi_z_2],
              plot=True
              )

Full tree

In [ ]:
print("Full tree")
scores = knn_purity(max_neighbors=50,
              data=[glm.z, full_scvi_latent, full_scvi_latent_2, full_cascvi_latent, full_cascvi_latent_2],
              plot=True)

***Stratified k-NN purity***

In [ ]:
for k in [2, 5, 10, 20, 35, 50]:
    print("For {} neighbors".format(k))
    if k == 10:
        min_depth = 3
    elif k == 20:
        min_depth = 4
    elif k == 35:
        min_depth = 6
    elif k == 50:
        min_depth = 7
    else:
        min_depth = 2
    scores = knn_purity_stratified(n_neighbors=k,
                                   tree=tree,
                                   data=[glm.z, full_scvi_latent, full_scvi_latent_2, full_cascvi_latent, full_cascvi_latent_2],
                                   min_depth=min_depth,
                                   plot=True)

# 7. Variance analysis

In [ ]:
# Get empirical variance of the encoder
qz_v_norm = full_posterior.empirical_qz_v(
                                    n_samples=1000,
                                    norm=True)

***Empirical Variance of observed***

In [ ]:
var_dict = {}
node_sizes = []

for i, n in enumerate(tree.traverse('levelorder')):
    if not n.is_leaf():
        # if n is an internal node, we get the variance from the prior
        z = full_posterior.get_latent(give_mean=False)[0]
        _, v_nu = vae.posterior_predictive_density(query_node=n.name, 
                                                  evidence=z) 
        
        var_dict[n.name] = str(np.around(a=v_nu,
                             decimals=4)
                              )
        node_sizes.append(np.around(a=v_nu,
                             decimals=4)
                         )
    else:
        idx = leaves_idx.index(n.index)
        # if n is a leaf, we get the variance from the encoder
        
        # variational distribution variance
        
        # Empirical variance
        var_dict[n.name] = str(np.around(a=qz_v_norm[idx],
                                 decimals=4)
                             )
        node_sizes.append(np.around(a=qz_v_norm[idx],
                                 decimals=4) 
                         )

In [ ]:
var_dict

***Plot variance***

In [ ]:
plot_one_gene(tree=tree, 
              X=imputed_X, 
              g=0, 
              node_sizes=[int(n*10000) for n in node_sizes], #+[1000]
              var='latent',
              size=1000,
              show_index=True,
              save_fig=True,
              figsize=(180, 100)
             )

print("CascVI: Imputations + Variance analysis")

# 7. Evaluation

***CPM Normalization (for sample-sample correlation)***

get imputations into an array

In [200]:
#internal_scvi_X_2 = np.array([x for x in imputed_scvi_2.values()]).reshape(-1, glm.X.shape[1])
internal_cascvi_X_2 = np.array([x for x in imputed_cascvi_2.values()]).reshape(-1, glm.X.shape[1])

internal_cascvi_X_2.shape

(72, 100)

In [201]:
from sklearn.preprocessing import normalize

norm_internal_X = sc.pp.normalize_total(AnnData(internal_X), target_sum=1e4, inplace=False)['X'] 
#norm_scvi_X_2 = sc.pp.normalize_total(AnnData(internal_scvi_X_2), target_sum=1e4, inplace=False)['X']
norm_avg_X = sc.pp.normalize_total(AnnData(internal_avg_X), target_sum=1e4, inplace=False)['X']
norm_imputed_X = sc.pp.normalize_total(AnnData(imputed_X), target_sum=1e4, inplace=False)['X']
norm_cascvi_X_2 = sc.pp.normalize_total(AnnData(internal_cascvi_X_2), target_sum=1e4, inplace=False)['X']

norm_internal_X.shape

(72, 100)

## I. Sample-Sample Correlations

***1. Sample-Sample correlation (Without Normalization)***

We will use Scipy to compute a nonparametric rank correlation between the imputed and the groundtruth profiles. The correlation is based on the Spearman Correlation Coefficient.

In [203]:
data = {'groundtruth':internal_X.T, 'cascVI':imputed_X.T, 
        'Average':internal_avg_X.T , 'cascVI + Avg': internal_cascvi_X_2.T
        }
df1 = correlations(data, 'None', True)
df1.head(5)
plt.show()

/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


***2. Sample-Sample correlation (With ScanPy Normalization)***

In [206]:
data = {'groundtruth': norm_internal_X.T, 'cascVI': norm_imputed_X.T, 
        'Average': norm_avg_X.T , 'cascVI + Avg': norm_cascvi_X_2.T
        }

df2 = correlations(data, 'None', True)
df2.head(5)
plt.show()

/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())



## II. Gene-Gene Correlations

***2. Gene-Gene correlation (With Normalization)***

In [ ]:
data = {'groundtruth': norm_internal_X, 'cascVI': norm_imputed_X, 
        'Average': norm_avg_X , 'cascVI + Avg': norm_cascvi_X_2
        }

df4 = correlations(data, 'None', True)
df4.head(5)
plt.show()

***3. Gene-Gene correlation (With Rank Normalization)***

In [209]:
data = {'groundtruth': norm_internal_X, 'cascVI': norm_imputed_X, 
        'Average': norm_avg_X , 'cascVI + Avg': norm_cascvi_X_2
        }

df5 = correlations(data, 'rank', True)
df5.head(5)
plt.show()

/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


### III. Table Summary

In [ ]:
columns = ["Method", "Spearman CC", "Pearson CC", "Kendall Tau"]
data = [df1, df2, df3, df4, df5]

tables = [[] for i in range(len(data))]

#task = ["Sample-Sample (None)", "Sample-Sample (CPM)", "Gene-Gene (None)", 
           #"Gene-Gene(CPM)", "Gene-Gene (Rank)" ]

for (df, t) in zip(data, tables):
    for m in np.unique(df.Method):
        sub_df = np.round(df[df['Method'] == m].mean(), decimals=3)
        t.append([m, sub_df['Spearman CC'], sub_df['Pearson CC'], sub_df['Pearson CC']])
        
# Create and style Data Frames
df_table1 = pd.DataFrame(tables[0], columns=columns)
df_table2 = pd.DataFrame(tables[1], columns=columns)
df_table3 = pd.DataFrame(tables[2], columns=columns)
df_table4 = pd.DataFrame(tables[3], columns=columns)
df_table5 = pd.DataFrame(tables[4], columns=columns)

In [ ]:
print(" >>> Sample-Sample | No Normalization <<<")
df_table1.head(10)

In [ ]:
print(">>> Sample-Sample | CPM Normalization <<<")
df_table2.head(10)

In [ ]:
print(">>> Gene-Gene | No Normalization <<<")
df_table3.head(10)

In [ ]:
print(">>> Gene-Gene | CPM Normalization <<<")
df_table4.head(10)

In [ ]:
print(">>> Gene-Gene | Rank Normalization <<<")
df_table5.head(10)

# 8. Exploratory grapghical analysis

***Box plot (Samples)***

In [ ]:
import random
import seaborn as sns

n_samples = 10
idx = random.sample(range(1, 300), n_samples)
n_rows = 2
n_cols = int(n_samples / 2)

fig, ax = plt.subplots(n_rows, n_cols, figsize=(30, 12))

h = 0
for i in range(n_rows):
    for j in range(n_cols):
        k = idx[h]
        h += 1
        # stack arrrays
        data_boxplot = np.vstack([np.log(1 + norm_internal_X[k]), np.log(1 + norm_imputed_X[k]),
          np.log(1 + norm_avg_X[k]), np.log(1 + norm_scvi_X[k]),
           np.log(1 + norm_cascvi_X[k])]).T
        
        # data frame
        df_boxplot = pd.DataFrame(data=data_boxplot,
                          columns=['Ground truth', 'CascVI', 'Avg', 'Avg scVI', 'Avg CascVI'])
        
        sns.boxplot(ax=ax[i][j], data=df_boxplot, orient="v", palette="Set2", showfliers=True)
        ax[i][j].set_title('Sample ' + str(k))

fig.suptitle("Boxplots of gene expression values")
plt.show()

***Box plot (Genes)***

In [ ]:
import random
import seaborn as sns

n_samples = 10
idx = random.sample(range(1, 1000), n_samples)
n_rows = 2
n_cols = int(n_samples / 2)

fig, ax = plt.subplots(n_rows, n_cols, figsize=(30, 12))

h = 0
for i in range(n_rows):
    for j in range(n_cols):
        k = idx[h]
        h += 1
        # stack arrrays
        data_boxplot = np.vstack([np.log(1 + norm_internal_X[:, k]), np.log(1 + norm_imputed_X[:, k]),
          np.log(1 + norm_avg_X[:, k]), np.log(1 + norm_scvi_X[:, k]),
           np.log(1 + norm_cascvi_X[:, k])]).T
        
        # data frame
        df_boxplot = pd.DataFrame(data=data_boxplot,
                          columns=['Ground truth', 'CascVI', 'Avg', 'Avg scVI', 'Avg CascVI'])
        
        sns.boxplot(ax=ax[i][j], data=df_boxplot, orient="v", palette="Set2", showfliers=True)
        ax[i][j].set_title('Gene ' + str(k))

fig.suptitle("Boxplots of gene expression values")
plt.show()

***Density plots***

In [ ]:
import random
import seaborn as sns

n_samples = 12
idx = random.sample(range(1, 1000), n_samples)
n_rows = 3
n_cols = int(n_samples / n_rows)

fig, ax = plt.subplots(n_rows, n_cols, figsize=(35, 15))

h = 0
for i in range(n_rows):
    for j in range(n_cols):
        k = idx[h]
        h += 1
        # density plots
        sns.distplot(ax=ax[i][j], a=np.log(1 + norm_imputed_X[:, k]), hist=False,
                     kde=True, kde_kws={'shade': True}, label='cascVI')
        sns.distplot(ax=ax[i][j], a=np.log(1 + norm_internal_X[:, k]), hist=False,
                     kde=True, kde_kws={'shade': True}, label='groundtruth')
        sns.distplot(ax=ax[i][j], a=np.log(1 + norm_scvi_X[:, k]), hist=False,
                     kde=True, kde_kws={'shade': True}, label='Avg scVI')
        sns.distplot(ax=ax[i][j], a=np.log(1 + norm_avg_X[:, k]), hist=False,
                     kde=True, kde_kws={'shade': True}, label='Avg')
        sns.distplot(ax=ax[i][j], a=np.log(1 + norm_cascvi_X[:, k]), hist=False,
             kde=True, kde_kws={'shade': True}, label='Avg cascVI')
        # set title
        ax[i][j].set_title('Gene ' + str(k))

fig.suptitle("Combined gene density plots")
plt.show()